In [1]:
import os
from glob import glob

import numpy as np
import pandas as pd

In [2]:
def export_parameter_record(savefolder, params_dict):

    import os

    for i in range(1, 1000):
        savepath = os.path.join(savefolder, "Param_Record_{}.txt".format(str(i).zfill(3)))
        if os.path.exists(savepath):
            continue
        else:
            break

    with open(savepath, "w+") as file:
        for key, value in params_dict.items():
            file.write("{} : {}".format(key, value))
            file.write("\n\n")

    pop_up_string = "Saved at {}".format(savefolder)
    print(pop_up_string)


def open_geodata(filepath):
    import sys

    import geopandas as gpd

    geodata_type = type(gpd.GeoDataFrame())

    if type(filepath) == geodata_type:
        geodata = filepath
    elif type(filepath) == str:
        extension = filepath.split(".")[-1]
        if extension == "pkl":
            geodata = pd.read_pickle(filepath, compression="zip")
        elif extension == "shp":
            geodata = gpd.read_file(filepath)
        else:
            sys.exit(
                "The datatype of PSC_filepath is invalid.\
            \nExpect {} or {}, not {}".format(
                    type("1"), geodata_type, type(PSC_filepath)
                )
            )
    else:
        sys.exit("{} invalid filetype".format(type(filepath)))
    return geodata

#### Data Input

In [3]:
profile_segment_psc_folder = r"E:\010__SS3_TRY_MY_BEST_TO_SURVIVE_POSTPROCESSING\HSR_PROFILE_Manuscript\CROSSTrackProfile\CR_PSC_GeoData"

file2process = glob(os.path.join(profile_segment_psc_folder, "Output_*"))

savefolder = r"E:\010__SS3_TRY_MY_BEST_TO_SURVIVE_POSTPROCESSING\HSR_PROFILE_Manuscript\CROSSTrackProfile\CR_PSC_CSV_Export"

params_dict = {
    "profile_segment_psc_folder": profile_segment_psc_folder,
    "file2process": file2process,
    "savefolder": savefolder,
}

#### Function Processing

In [4]:
for file in file2process:

    data = open_geodata(file)

    unique_linename = data["LineName"].unique()

    for line_name in unique_linename:

        select_subdata = data[data["LineName"] == line_name]

        select_subdata = select_subdata.iloc[:, 2:-1]

        select_subdata = select_subdata.drop("LineName", axis=1)

        unique_segment = np.unique(data["Segment"].values)

        amount_list = []

        empty_mean = pd.DataFrame(data=None)
        empty_stdev = pd.DataFrame(data=None)

        for segment_name in unique_segment:

            data_by_segment = select_subdata[select_subdata["Segment"] == segment_name]
            average_by_segment = data_by_segment.groupby("Segment").mean()
            stdev_by_segment = data_by_segment.groupby("Segment").std()

            amount_list.append(len(data_by_segment))

            empty_mean = pd.concat([empty_mean, average_by_segment])
            empty_stdev = pd.concat([empty_stdev, stdev_by_segment])

        empty_mean["Pts_Num"] = amount_list
        empty_stdev["Pts_Num"] = amount_list

        savename_mean = "{}_{}_{}.xlsx".format("Mean", os.path.basename(file)[:-4], line_name)
        savename_stdev = "{}_{}_{}.xlsx".format("Stdev", os.path.basename(file)[:-4], line_name)

        empty_mean.to_excel(os.path.join(savefolder, savename_mean))
        empty_stdev.to_excel(os.path.join(savefolder, savename_stdev))

export_parameter_record(savefolder=savefolder, params_dict=params_dict)

Saved at E:\010__SS3_TRY_MY_BEST_TO_SURVIVE_POSTPROCESSING\HSR_PROFILE_Manuscript\CROSSTrackProfile\CR_PSC_CSV_Export
